<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/Fake_News_Classification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [1]:
import pandas as pd
import os

In [22]:
!unzip /content/train_LSTM.zip

Archive:  /content/train_LSTM.zip
  inflating: train_LSTM.csv          


In [23]:
df=pd.read_csv('/content/train_LSTM.csv')

In [24]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [25]:
df.shape

(20800, 5)

In [26]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [27]:
df=df.dropna()

In [28]:
df.reset_index(drop=True,inplace=True)

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
X=df.drop('label',axis=1)
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
X.drop('id',axis=1,inplace=True)

In [2]:
import tensorflow as tf


In [3]:
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
messages1=X.copy()

## DATA CLEANING


In [ ]:
def tolower(text):
  return text.lower()

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(tolower)

In [ ]:
messages1.head()

,title,author,text
0,house dem aide: we didn’t even see comey’s let...,darrell lucus,house dem aide: we didn’t even see comey’s let...
1,"flynn: hillary clinton, big woman on campus - ...",daniel j. flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnews.com,"why the truth might get you fired october 29, ..."
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [ ]:
## REMOVING HTML TAGS
import re
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_html_tags)

In [ ]:
## removing url
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_url)

In [ ]:
## HANDLING PUNCTUATION
import string
def remove_punc(text):
  for i in string.punctuation:
    text=text.replace(i,'')
  return text


In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_punc)

In [ ]:
messages1.head()

,title,author,text
0,house dem aide we didn’t even see comey’s lett...,darrell lucus,house dem aide we didn’t even see comey’s lett...
1,flynn hillary clinton big woman on campus bre...,daniel j flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnewscom,why the truth might get you fired october 29 2...
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [ ]:
## REMOVING STOPWORDS
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_stopwords(text):
  words=text.split()
  for i,word in enumerate(words):
    if word in stopwords.words('english'):
      words[i]=''
  return ' '.join(words)

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_stopwords)

In [ ]:
messages1['title'][2]

'  truth might get  fired'

In [ ]:
def remove_extra_spaces(sentence):

    cleaned_sentence = ' '.join(sentence.split()).strip()

    return cleaned_sentence

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_extra_spaces)

In [ ]:
messages1['title'][2]

'truth might get fired'

In [ ]:
messages1.to_csv('cleaned_data.csv')

In [4]:
!unzip /content/cleaned_data.zip

Archive:  /content/cleaned_data.zip
  inflating: cleaned_data.csv        


In [5]:
msg=pd.read_csv('/content/cleaned_data.csv')

In [6]:
msg=msg['title']

In [7]:
msg

0        house dem aide didn’t even see comey’s letter ...
1         flynn hillary clinton big woman campus breitbart
2                                    truth might get fired
3        15 civilians killed single us airstrike identi...
4        iranian woman jailed fictional unpublished sto...
                               ...                        
18280       rapper ti trump ’poster child white supremacy’
18281    nfl playoffs schedule matchups odds new york t...
18282    macy’s said receive takeover approach hudson’s...
18283          nato russia hold parallel exercises balkans
18284                                      keeps f35 alive
Name: title, Length: 18285, dtype: object

In [8]:
sentences=list(msg)

In [9]:
sentences

['house dem aide didn’t even see comey’s letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 '15 civilians killed single us airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathrooms exclusive video breitbart',
 'benoît hamon wins french socialist party’s presidential nomination new york times',
 'backchannel plan ukraine russia courtesy trump associates new york times',
 'obama’s organizing action partners soroslinked ‘indivisible’ disrupt trump’s agenda',
 'bbc comedy sketch real housewives isis causes outrage',
 'russian researchers discover secret nazi military base ‘treasure hunter’ arctic photos',
 'us officials see link trump russia',
 'yes paid government trolls social media blogs forums websites',
 'major league soccer argentines find home success new york times',
 'wells fargo chief abruptly 

In [10]:
sentences = [str(sentence) for sentence in sentences]

In [11]:
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 5000   #vocabulary size
onehot_repe = [one_hot(i,voc_size)for i in sentences]      #The one_hot function will encode words into integers, and the integers will range from 0 to voc_size - 1.
onehot_repe

[[4950, 37, 2553, 561, 3478, 2148, 250, 3940, 4644, 863, 2583],
 [4538, 1421, 4481, 1447, 3764, 2796, 2134],
 [564, 3253, 3713, 4798],
 [4456, 140, 3320, 4206, 3446, 2622, 3568],
 [3553, 3764, 2497, 540, 2826, 2490, 3764, 940, 2462, 3106],
 [4333,
  1443,
  2425,
  4568,
  4446,
  3223,
  1816,
  4372,
  2868,
  810,
  4537,
  1471,
  2892,
  1821,
  2134],
 [1771, 32, 4429, 431, 461, 2357, 844, 801, 648, 728, 1119],
 [3879, 4732, 1584, 1775, 3669, 3223, 273, 648, 728, 1119],
 [2528, 3929, 398, 1060, 2103, 2783, 1669, 827, 1205],
 [3572, 2370, 4654, 414, 1020, 3304, 1536, 667],
 [2289, 2377, 1457, 4083, 3192, 2336, 4855, 3684, 4412, 3015, 3261],
 [3446, 4751, 2148, 2883, 3223, 1775],
 [4828, 4244, 4093, 225, 1773, 4399, 2399, 2605, 2355],
 [1549, 3472, 3063, 424, 3519, 492, 4918, 648, 728, 1119],
 [2076, 533, 1028, 1526, 3777, 648, 728, 1119],
 [100, 3006, 4188, 910, 3485, 1394, 1302, 608, 2861, 1725, 2043],
 [4692, 735, 1421],
 [619, 3653, 4901, 2795, 3223, 1360, 1783, 2134],
 [3190, 

In [12]:
onehot_repe[0]

[4950, 37, 2553, 561, 3478, 2148, 250, 3940, 4644, 863, 2583]

In [17]:
sentences[0]

'house dem aide didn’t even see comey’s letter jason chaffetz tweeted'

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 50
embedded_doc = pad_sequences(onehot_repe, maxlen = sent_length, padding = 'pre')

In [15]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 4950,   37, 2553,  561, 3478,
       2148,  250, 3940, 4644,  863, 2583], dtype=int32)

## CREATING MODEL

In [16]:
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310501 (1.18 MB)
Trainable params: 310501 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
from sklearn.model_selection import train_test_split
import numpy as np

In [31]:
X_input=np.array(embedded_doc)
y_input=np.array(y)

In [33]:
X_input.shape

(18285, 50)

In [34]:
y_input.shape

(18285,)

In [35]:
X_train,X_test,y_train,y_test=train_test_split(X_input,y_input,test_size=0.33,random_state=42)

In [37]:
# training
model.fit(X_train,y_train,epochs=5,batch_size=64,validation_data=(X_test,y_test))

Epoch 1/5
192/192 [==============================] - 20s 90ms/step - loss: 0.3219 - accuracy: 0.8461 - val_loss: 0.1975 - val_accuracy: 0.9176
Epoch 2/5
192/192 [==============================] - 17s 90ms/step - loss: 0.1251 - accuracy: 0.9510 - val_loss: 0.2186 - val_accuracy: 0.9100
Epoch 3/5
192/192 [==============================] - 17s 90ms/step - loss: 0.0715 - accuracy: 0.9746 - val_loss: 0.2320 - val_accuracy: 0.9122
Epoch 4/5
192/192 [==============================] - 18s 91ms/step - loss: 0.0370 - accuracy: 0.9882 - val_loss: 0.3294 - val_accuracy: 0.9133
Epoch 5/5
192/192 [==============================] - 16s 85ms/step - loss: 0.0169 - accuracy: 0.9956 - val_loss: 0.4237 - val_accuracy: 0.9130


In [38]:
# lets check how it is performing
y_pred = (model.predict(X_test)>0.5).astype("int64")
'''
After making predictions, this part of the code compares each prediction with 0.5.
The result is a Boolean array where each element is True if the corresponding prediction is greater than 0.5 and False otherwise.
Finally, the code converts the Boolean array obtained from the previous step to an array of integers (int64).
'''

189/189 [==============================] - 4s 15ms/step


'\nAfter making predictions, this part of the code compares each prediction with 0.5.\nThe result is a Boolean array where each element is True if the corresponding prediction is greater than 0.5 and False otherwise.\nFinally, the code converts the Boolean array obtained from the previous step to an array of integers (int64).\n'

In [39]:
y_pred.shape

(6035, 1)

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [41]:
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 232, 2384]])

In [42]:
accuracy_score(y_test,y_pred)

0.9130074565037283